<a href="https://colab.research.google.com/github/edwintenkorang/PSO-A2C-Lnet/blob/main/Cleaned_Up_Code_for_PSO_A2C_LNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Import the data from Google Drive
from google.colab import drive
import pandas as pd
drive.mount('/content/drive')

In [ ]:
#'/content/drive/My Drive/Data for PSO-A2C-LNet/your_file.csv'
file_path_train = '/content/drive/MyDrive/Data for PSO-A2C-LNet/Panama Dataset/train_dataframes.xlsx'
file_path_test = '/content/drive/MyDrive/Data for PSO-A2C-LNet/Panama Dataset/test_dataframes.xlsx'
import pandas as pd
data = pd.read_excel(file_path_train)
data_y = pd.read_excel(file_path_test)

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
import io
import pandas as pd
data = pd.read_excel(io.BytesIO(uploaded['train_dataframes (1).xlsx']))
# Dataset is now stored in a Pandas Dataframe

In [ ]:
#Installations
!pip install pyswarm

In [ ]:
#BEST ACCURACY SO FAR FOR THE PANAMA == 97.1903%
#A2C-LNet
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv1D, Dropout, LSTM, Dense, Bidirectional, Activation, Multiply
from tensorflow.keras.optimizers import Adam
import tensorflow as tf

# Data Preprocessing
features = data.drop(columns=['datetime', 'DEMAND']).values
target = data['DEMAND'].values

# Normalize features
scaler = MinMaxScaler()
features = scaler.fit_transform(features)

window_size = 4
X, y = [], []
for i in range(len(features) - window_size + 1):
    X.append(features[i:i + window_size])
    y.append(target[i + window_size - 1])

X = np.array(X)
y = np.array(y)

# Reshape X for the Conv1D layer
X = X.reshape(X.shape[0], X.shape[1], X.shape[2])

# Step 2: Splitting the Data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

input_shape = (window_size, features.shape[1])

input_layer = Input(shape=input_shape)
conv_layer = Conv1D(filters=64, kernel_size=4, activation='relu')(input_layer)
dropout1_layer = Dropout(0.2)(conv_layer)
lstm_layer = Bidirectional(LSTM(128, activation='tanh'))(dropout1_layer)
dropout2_layer = Dropout(0.2)(lstm_layer)

# Attention Module
attention_layer = Dense(1, activation='tanh')(dropout2_layer)
attention_layer = Activation('softmax')(attention_layer)
attention_mul_layer = Multiply()([dropout2_layer, attention_layer])

output_layer = Dense(1, activation='linear')(attention_mul_layer)  # Output layer with linear activation for regression

# Model Compilation
model = Model(inputs=input_layer, outputs=output_layer)
model.compile(optimizer=Adam(), loss='mean_absolute_percentage_error')
model.summary()

# Train the Model
model.fit(X_train, y_train, epochs=3500, batch_size=16)

# Evaluate the Model
y_pred = model.predict(X_test).flatten()

# Check if the arrays have the same shape
if y_test.shape != y_pred.shape:
    raise ValueError("Shape mismatch between y_test and y_pred")

# Calculate R^2
def r_squared(y_true, y_pred):
    residual = tf.reduce_sum(tf.square(tf.subtract(y_true, y_pred)))
    total = tf.reduce_sum(tf.square(tf.subtract(y_true, tf.reduce_mean(y_true))))
    r2 = 1 - tf.divide(residual, total)
    return r2

r2 = r_squared(y_test, y_pred)
print("R^2 Score:", r2.numpy())

# MAE
mae = tf.reduce_mean(tf.abs(y_test - y_pred))
print("Mean Absolute Error (MAE):", mae.numpy())

# MAPE
mape = tf.reduce_mean(tf.abs((y_test - y_pred) / y_test)) * 100
print("Mean Absolute Percentage Error (MAPE):", mape.numpy())

In [ ]:
#BEST ACCURACY SO FAR FOR THE PANAMA == 98.0624%
#Adding PSO to the A2C-LNet

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv1D, Dropout, LSTM, Dense, Bidirectional, Activation, Multiply
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
from pyswarm import pso

# Define the objective function for PSO
def optimize_hyperparameters(params):
    learning_rate, batch_size, num_epochs, weight_init, loss_metric = params

    # Data Preprocessing (Assuming 'data' is loaded previously)
    features = data.drop(columns=['datetime', 'DEMAND']).values
    target = data['DEMAND'].values

    # Normalize features
    scaler = MinMaxScaler()
    features = scaler.fit_transform(features)

    window_size = 4
    X, y = [], []
    for i in range(len(features) - window_size + 1):
        X.append(features[i:i + window_size])
        y.append(target[i + window_size - 1])

    X = np.array(X)
    y = np.array(y)

    # Reshape X for the Conv1D layer
    X = X.reshape(X.shape[0], X.shape[1], X.shape[2])

    # Splitting the Data
    X_train, _, y_train, _ = train_test_split(X, y, test_size=0.2, random_state=42)  # Training data

    # Assuming 'data_y' contains the testing dataset
    features_y = data_y.drop(columns=['datetime', 'DEMAND']).values
    target_y = data_y['DEMAND'].values

    # Normalize features for the testing dataset
    features_y = scaler.transform(features_y)

    X_y, y_y = [], []
    for i in range(len(features_y) - window_size + 1):
        X_y.append(features_y[i:i + window_size])
        y_y.append(target_y[i + window_size - 1])

    X_y = np.array(X_y)
    y_y = np.array(y_y)

    # Reshape X for the Conv1D layer for the testing dataset
    X_y = X_y.reshape(X_y.shape[0], X_y.shape[1], X_y.shape[2])

    # Define initializer
    initializer = None

    input_shape = (window_size, features.shape[1])

    input_layer = Input(shape=input_shape)
    conv_layer = Conv1D(filters=64, kernel_size=4, activation='relu')(input_layer)
    dropout1_layer = Dropout(0.2)(conv_layer)
    lstm_layer = Bidirectional(LSTM(128, activation='tanh'))(dropout1_layer)
    dropout2_layer = Dropout(0.2)(lstm_layer)

    # Create a custom initializer for the dense layers
    dense_initializer = tf.keras.initializers.get(initializer)

    # Use the custom initializer in the Dense layers
    # Attention Module
    attention_layer = Dense(1, activation='tanh', kernel_initializer=dense_initializer)(dropout2_layer)
    attention_layer = Activation('softmax')(attention_layer)
    attention_mul_layer = Multiply()([dropout2_layer, attention_layer])

    output_layer = Dense(1, activation='linear', kernel_initializer=dense_initializer)(attention_mul_layer)

    if weight_init == 0:
        initializer = 'glorot_uniform'  # Xavier initialization
    elif weight_init == 1:
        initializer = 'he_normal'  # He initialization
    else:
        initializer = 'random_normal'  # Random initialization

    loss_metric = round(loss_metric)
    weight_init = round(weight_init)

    if loss_metric == 0:
      loss_metric = 'mean_squared_error'
    elif loss_metric == 1:
      loss_metric = 'binary_crossentropy'
    else:
      loss_metric = 'mean_absolute_percentage_error'

    num_epochs = round(num_epochs+0.5)
    batch_size = round(batch_size+0.5)

    # Model Compilation
    model = Model(inputs=input_layer, outputs=output_layer)
    model.compile(optimizer=Adam(learning_rate), loss=loss_metric, metrics=['mape'])

    # Train the Model
    model.fit(X_train, y_train, epochs=num_epochs, batch_size=batch_size, verbose=1)

    # Evaluate the Model on the testing dataset
    y_pred = model.predict(X_y).flatten()

    # Calculate MAPE for the testing dataset
    mape = tf.reduce_mean(tf.abs((y_y - y_pred) / y_y)) * 100

    return mape.numpy()

# Parameter ranges for PSO
lb = [0.001, 1, 100, 0, 0]  # Lower bounds for learning_rate, batch_size, num_epochs, weight_init, loss_metric
ub = [0.1, 128, 5000, 2, 2]  # Upper bounds for learning_rate, batch_size, num_epochs, weight_init, loss_metric

# Perform PSO optimization
best_params, _ = pso(optimize_hyperparameters, lb, ub, swarmsize=1200, maxiter=1000)

# Print the best hyperparameters
learning_rate, batch_size, num_epochs, weight_init, loss_metric = best_params
print("Best Hyperparameters:")
print("Learning Rate:", learning_rate)
print("Batch Size:", int(batch_size))
print("Number of Epochs:", int(num_epochs))
if weight_init == 0:
    print("Weight Initialization: Xavier")
elif weight_init == 1:
    print("Weight Initialization: He")
else:
    print("Weight Initialization: Random")
if loss_metric == 0:
    print("Loss Metric: Mean Squared Error (MSE)")
elif loss_metric == 1:
    print("Loss Metric: Cross-Entropy")
else:
    print("Loss Metric: Mean Absolute Percentage Error (MAPE)")